In [1]:
import os
import sys

# Add library path to Python path
sys.path.append(os.path.abspath('..'))

# Now import your modules
from library import libStopsPoints, libConnections, libHex
import zipfile
import os
import time
import pymongo as pym
import pandas as pd
import folium
import numpy as np
import requests
import numba

import math
import geopy
from shapely.geometry import Polygon, MultiPolygon, Point, mapping
from geopy.distance import geodesic,great_circle
from folium.plugins import FastMarkerCluster
from datetime import datetime
from geopy.distance import geodesic,great_circle
from pymongo import MongoClient

from pathlib import Path
import geopandas as gpd
import geojson
from IPython.core.display import display, HTML

from library import libStopsPoints, libConnections, libHex
 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11864\3252682998.py:31: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Url and paths  [**set it!**]

In [2]:
city = 'padam_territory_01'
scenario_name = 'basecase' # basecase without drt

# file paths and addresses to be provided:
# population data
shpPath = './shp/poi.shp'
popCollectionName = "POP"
popField = "pop"

# Study area
# Study area defines the area in which the accessibility calaculation is executes
# study_area_shp_path = 'Paris/shp/studyarea.shp'


directoryGTFS = './gtfs/'

gtfs_prep_toggle = True
urlMongoDb = "mongodb://localhost:27017/"  # url of the mongodb database
urlMongoDbPop = "mongodb://localhost:27017/" # url of the mongodb database for population
urlServerOsrm = 'http://localhost:5000/' # url of the osrm server of the city

client = MongoClient(urlMongoDb)

gtfsDB = client[str('pta-' + city + '-' +scenario_name)]
popDbName = str('pta-' + city + '-' +scenario_name)

# required parameters
# the date must be in the interval of validity of the gtfs files, check it in the "calendar.txt" and "calendar_dates.txt" files inside the gtfs zip files.
# the day for the computation of accessibility quantitites
days = ['20220307']
day_names = [ "monday"]


# parameters of walking distance
timeWalk = 15 * 60  # seconds
velocityWalk = 1.39  # m/s ***4.71km/h***
distanceS = timeWalk * velocityWalk

# Parameters that define the resolution and extention of tesselletion and the maximum of the walking time
# grid step of the hexagonal tesselletion in kilometers
gridEdge = 1

# Set check4stops = False if cells / hexagones should be included that do not have stops within.
# Set check4stops = False for preprocessing prior to dynamic mode to gtfs convertion
# Set check4stops = True for citychrone accessibility analysis
check4stops = False

# Start of the computation

### add population data

In [6]:

shapefile = gpd.read_file(shpPath)
shapefile.to_file("./shp/poi.geojson", driver='GeoJSON')
with open("./shp/poi.geojson",encoding="utf8") as f:
    gj = geojson.load(f)
features = gj['features']
gtfsDB["POP"].drop()
gtfsDB["POP"].insert_many(features)


InsertManyResult([ObjectId('67d01c98e1764feeace9be8e'), ObjectId('67d01c98e1764feeace9be8f'), ObjectId('67d01c98e1764feeace9be90'), ObjectId('67d01c98e1764feeace9be91'), ObjectId('67d01c98e1764feeace9be92'), ObjectId('67d01c98e1764feeace9be93'), ObjectId('67d01c98e1764feeace9be94'), ObjectId('67d01c98e1764feeace9be95'), ObjectId('67d01c98e1764feeace9be96'), ObjectId('67d01c98e1764feeace9be97'), ObjectId('67d01c98e1764feeace9be98'), ObjectId('67d01c98e1764feeace9be99'), ObjectId('67d01c98e1764feeace9be9a'), ObjectId('67d01c98e1764feeace9be9b'), ObjectId('67d01c98e1764feeace9be9c'), ObjectId('67d01c98e1764feeace9be9d'), ObjectId('67d01c98e1764feeace9be9e'), ObjectId('67d01c98e1764feeace9be9f'), ObjectId('67d01c98e1764feeace9bea0'), ObjectId('67d01c98e1764feeace9bea1'), ObjectId('67d01c98e1764feeace9bea2'), ObjectId('67d01c98e1764feeace9bea3'), ObjectId('67d01c98e1764feeace9bea4'), ObjectId('67d01c98e1764feeace9bea5'), ObjectId('67d01c98e1764feeace9bea6'), ObjectId('67d01c98e1764feeace9be

In [3]:
# load Gtfs data to mongodb database
listOfFile = ['stops.txt', 'routes.txt', 'trips.txt', 'calendar.txt', 'calendar_dates.txt',
              'stop_times.txt']  
libStopsPoints.loadGtfsFile(gtfsDB, directoryGTFS, city, listOfFile)

removing stops  of  padam_territory_01
removing routes  of  padam_territory_01
removing trips  of  padam_territory_01
removing calendar  of  padam_territory_01
removing calendar_dates  of  padam_territory_01
removing stop_times  of  padam_territory_01
gtfs.zip
stops.txt -> (400,400)
routes.txt -> (71,71)
trips.txt -> (5797,5797)
calendar.txt -> (74,74)
calendar_dates.txt -> (83,83)
stop_times.txt -> (22670,22670)


## Fill the database with the connections

In [4]:

libConnections.readConnections(gtfsDB, city, directoryGTFS, days[0], day_names[0], overwrite=True)


number of file in calendar+calendar_dates: 1
in stops: 1

Checking the number of services active in the date selected:
file: gtfs.zip 	 total number of active service (in calendar.txt): 34
number of different service_id: 1


file: gtfs.zip 	 total number of active service (in calendar_dates.txt): 34
number of different service_id: 1 total number of active services found: 34
number of trips 5000
 gtfs.zip
inserting to DB....6, err 0, err_start 0, err_start_after 0
tot connections 9604


## Remove stops with no connections and add to each stop the pos field

In [5]:
libStopsPoints.removingStopsNoConnections(gtfsDB, city)
libStopsPoints.setPosField(gtfsDB, city)

In [6]:
libConnections.updateConnectionsStopName(gtfsDB, city)

connections deleted 0 pEnd 9604, totC 9604


# Tassel with exagons

### List of all stops

In [7]:
stopsList = libStopsPoints.returnStopsList(gtfsDB, city)

tot stop 253  stop error : 0


## Compute the box that include all stops
The edge of such box are enlarged by distanceS.

In [8]:

display(HTML('<h1>All stops of the public transport present in the gtfs files</h1>'))
bbox = libStopsPoints.boundingBoxStops(stopsList)
libStopsPoints.mapStops(bbox, stopsList)

In [23]:
bbox

[1.8266460206188384, 48.85046916816086, 2.0514705793811614, 48.928343831839136]

## Tassel the box with exagons.

In [9]:
hexBin, pointBin = libHex.hexagonalGrid(bbox, gridEdge, gtfsDB['stops'], distanceS, city,check4stops)

In [10]:

libHex.insertPoints(pointBin, city, gtfsDB)


## Find the hex with walkingTime less than timeWalk from a stops

In [11]:

libHex.pointsServed(gtfsDB, stopsList, urlServerOsrm, distanceS, timeWalk, city)

In [12]:
print("Number of hexagons: {0}".format(gtfsDB['points'].count_documents({'served':True, 'city':city})))

Number of hexagons: 148


## Setting field "pos" for points for performance

In [13]:
from library import libHex 
libHex.settingHexsPos(gtfsDB, city)

## Setting Population of Hexagons

In [15]:
import numpy as np
from shapely.geometry import Polygon, Point

def setHexsPop(gtfsDB, popCol, namePopField, city):
    tot = gtfsDB['points'].count_documents({'city': city})
    count = 0
    totPop = 0

    for hexagon in gtfsDB['points'].find({'city': city}):
        shapelyHex = Polygon(hexagon['hex']['coordinates'][0])
        hexagon['hex']['properties'] = {'pop': 0}
        findJson = {'geometry': {'$geoIntersects': {'$geometry': hexagon['hex']}}}

        for box in popCol.find(findJson):
            # Check if the geometry is a Point or Polygon
            if box['geometry']['type'] == 'Point':
                shapelyPoint = Point(box['geometry']['coordinates'])
                if shapelyHex.contains(shapelyPoint):
                    hexagon['hex']['properties']['pop'] += box['properties'][namePopField]
            elif box['geometry']['type'] == 'Polygon':
                shapelyBox = Polygon(box['geometry']['coordinates'][0])
                areaInter = shapelyBox.intersection(shapelyHex).area
       
                
                popHexBox = box['properties'][namePopField] * areaInter / shapelyBox.area
                hexagon['hex']['properties']['pop'] += popHexBox
            else:
                print("Unexpected geometry type:", box['geometry']['type'])
                continue

        count += 1
        totPop += hexagon['hex']['properties']['pop']
        gtfsDB['points'].update_one({'_id': hexagon['_id']}, {'$set': {'pop': hexagon['hex']['properties']['pop']}})
        print('{0:.1f}% , tot population: {1:.0f}, current hex: {2:.0f}'.format(100. * count / tot, totPop, hexagon['hex']['properties']['pop']), end="\r")

from library import libHex
if urlMongoDbPop != "" and popCollectionName != "":
    clientPop = pym.MongoClient(urlMongoDbPop)
    popDb = clientPop[popDbName]
    popCollection = popDb[popCollectionName]
    setHexsPop(gtfsDB, popCollection, popField, city)
else:
    print("Population NOT INSERTED!")

res = gtfsDB['points'].update_many({'pop': {'$exists': False}}, {'$set': {'pop': 0}})
print("n° of matched hexagons with population Polygons: {0} \n not matched: {1} (set to zero)".format(gtfsDB['points'].count_documents({'pop': {'$exists': True}}), res.modified_count))

n° of matched hexagons with population Polygons: 148 
 not matched: 0 (set to zero)


# Adding the walking time between stops and points

In [16]:
from library import libStopsPoints 
libStopsPoints.computeNeigh(gtfsDB, urlServerOsrm, distanceS, timeWalk,  city)

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


# Compute quantities and observable

TimeList is the list of starting time for computing the isochrones

In [17]:
timeList = [7,10,12,17,20]

hStart = timeList[0]*3600

### List of connections

In [18]:
from library import libConnections 
# arrayCC = libConnections.makeArrayConnections(gtfsDB, hStart, city)
arrayCC_day1 = libConnections.makeArrayConnections(gtfsDB, hStart, city, day='20220307')


start making connections array
done recover all cc 8865
converted
Num of connection 8865


### List of list of the points and stops neighbors

In [19]:
from library import libStopsPoints 
arraySP = libStopsPoints.listPointsStopsN(gtfsDB, city)

## Compute accessibility quantities

In [21]:
import imp
from library import libAccessibility, icsa
imp.reload(libAccessibility)
from library.icsa import computeAccessibilities
imp.reload(icsa)

listAccessibility = ['accessibility30min']

computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    
    pass
for timeStart in timeList:
    timeStart *= 3600
    hEnd  = timeStart+6600 #consider only connection that start at timeStart and end before next timeStart in the list timeList
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
   
    computeAccessibilities(
    city, timeStart,hEnd, arrayCC_day1, arraySP, gtfsDB, 
    computeIsochrone, timeStart/3600 == timeList[0], 
    day='20220307', 
    max_travel_time=1800, #30min 
    listAccessibility=listAccessibility
)

    

Time Isochrone Start: 7.0


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

Time Isochrone Start: 10.00min_20220307: 62.0, time to finish: 0.0h, 0.0m


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

Time Isochrone Start: 12.00min_20220307: 62.0, time to finish: 0.0h, 0.0m


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

Time Isochrone Start: 17.00min_20220307: 59.0, time to finish: 0.0h, 0.0m


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

Time Isochrone Start: 20.00min_20220307: 62.0, time to finish: 0.0h, 0.0m


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

## compute Average accessibility

In [30]:
from library.libStopsPoints import computeAverage
computeAverage(["accessibility30min_20220307"], gtfsDB, city)

In [ ]:
from library.libHex import reduceGeojsonInShellSubField
from IPython.core.display import display, HTML

field1 = 'accessibility30min_20220307'
field2 = 'avg'
color = ['#993404', "#f16913", "#fdae6b", '#74c476', '#31a354', '#006d2c', "#6baed6", "#4292c6", "#2171b5", '#08519c', '#f768a1', '#dd3497', '#ae017e', '#49006a'];
shell = [0., 2., 4., 5, 6., 7, 8., 9, 10., 11, 12., 13, 15, 17.];
#print ("number of hexs in total", gtfsDB['points'].find({field1:{'$exists':True}, 'city':city}).count())
res = reduceGeojsonInShellSubField(list(gtfsDB['points'].find({'city':city})), field1, field2, color, shell)
#res = showMapHexRedux(city, gtfsDB['points'], field = field, shell = shell, save=True)

display(HTML('<h1> Accessibility Score</h1>'))

res[1]

shell 0.0-2.0 -> 12 hexs
shell 2.0-4.0 -> 15 hexs
shell 4.0-5 -> 3 hexs
shell 5-6.0 -> 4 hexs
shell 6.0-7 -> 11 hexs
shell 7-8.0 -> 1 hexs
shell 8.0-9 -> 2 hexs
shell 9-10.0 -> 6 hexs
shell 10.0-11 -> 7 hexs
shell 11-12.0 -> 4 hexs
shell 12.0-13 -> 8 hexs
shell 13-15 -> 8 hexs
shell 15-17.0 -> 11 hexs


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11864\813394238.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
